In [1]:
import json 

dataset_path = './data'
annotations_file_path = dataset_path + '/' + 'train_all.json'

with open(annotations_file_path,'r') as f:
    dataset = json.loads(f.read())

In [2]:
import pandas as pd

raw_data = {'image_id': [],
           'class_name': [],
           'class_id': [],
           'x_min': [],
           'y_min': [],
           'w': [],
           'h': []}
df = pd.DataFrame(raw_data)

classes = ["UNKNOWN", "General trash", "Paper", "Paper pack", "Metal", "Glass",
           "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing"]

categories = dataset['categories']
annotations = dataset['annotations']
images = dataset['images']

all_row=[]
for annotation in annotations:
    file_name = images[annotation['image_id']]['file_name']
    row = [file_name , classes[annotation['category_id']],int(annotation['category_id']),
          annotation['bbox'][0],annotation['bbox'][1],annotation['bbox'][2],annotation['bbox'][3]]
    all_row.append(row)
    # file 이름 ,클래스이름 , 클래스id , x_min,y_min,w,h

df = pd.concat([df,pd.DataFrame(all_row,columns=df.columns)],ignore_index=True)
print(df)
df.to_csv('data.csv',index=False)

                   image_id   class_name  class_id  x_min  y_min      w      h
0      batch_01_vt/0002.jpg  Plastic bag       8.0  109.0  150.7  161.9  159.8
1      batch_01_vt/0002.jpg  Plastic bag       8.0  413.2  196.1   72.6   52.7
2      batch_01_vt/0002.jpg      Plastic       6.0   42.5  196.7   68.3   86.5
3      batch_01_vt/0002.jpg        Glass       5.0    0.1  279.7  117.3  183.0
4      batch_01_vt/0002.jpg        Glass       5.0  110.3   77.5   80.0  293.4
...                     ...          ...       ...    ...    ...    ...    ...
26395     batch_03/0997.jpg        Paper       2.0    0.0   97.2   28.9   82.9
26396     batch_03/0997.jpg        Paper       2.0    0.0  151.4   17.7   24.1
26397     batch_03/0997.jpg        Paper       2.0    0.0  261.0  112.8  147.7
26398     batch_03/0997.jpg        Paper       2.0    0.3  426.3   16.7   85.6
26399     batch_03/1000.jpg        Metal       4.0  258.7  158.3  124.9  272.0

[26400 rows x 7 columns]


In [3]:
class CFG:
    n_folds = 5
    seed = 42

# CFG.n_folds , CFG.seed 로 호출해서 사용가능

In [4]:
import random
import torch
import os

def seed_everything(seed=42):
    """Seed All

    Args:
        seed: seed number
    """
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True 

In [19]:
import numpy as np
import random
from collections import defaultdict

# 전체 annotation 개수만큼 df 행이 있음 , image_names는 중복되는 값들이 있음 , images_name과 category_id가 똑같은것도 여러개 있을수 있음(한 이미지에 똑같은 카테고리가 여러개 있을수 있으므로)
def stratified_k_fold(df,category_ids,image_names,k,seed=None):# df : df , category_ids : df['class_id'], image_names : df['image_id']
    labels_num = np.max(category_ids) + 1 # 11 (category_ids는 0부터 10까지 존재 -> 전체 개수는 11개)
    class_counts_per_image_names = defaultdict(lambda:np.zeros(labels_num))
    class_counts_total = defaultdict(int)
    
    for category_id,image_name in zip(category_ids,image_names):
        class_counts_per_image_names[image_name][category_id] += 1 # 한 이미지의 class 분포를 알아보기 위함
        class_counts_total[category_id] += 1 # 전체 class 분포를 알아보기 위함
        
    class_counts_per_fold = defaultdict(lambda:np.zeros(labels_num)) # fold 당 class 분포를 알기 위함 -> fold 별로 class를 고르게 분포하게 해야함
    image_names_per_fold = defaultdict(set) # fold 별로 image name을 분류하기 위함
    
    def eval_class_counts_per_fold(class_counts_image,fold): # 한 이미지의 class 분포를 보고 어떤 fold에 들어가면 좋을지 구하기 위하여 각 라벨들의 표준편차의 평균을 구함
        class_counts_per_fold[fold] += class_counts_image # fold 에 우선적으로 현재 이미지의 클래스 분포를 넣어보기
        # {0:[0,0,0,0,0,..,0],1:[0,0,0,0,...,0],...,4:[0,0,0,...,0]}
        std_per_class = [] # class 별로 표준편차를 구함
        for class_number in range(labels_num):
            # fold 별로 각 class 의 개수에서 전체 데이터에 대한 그 class의 개수를 나눠주고 그 값들의 표준편차를 구함
            # class_std 는 전체 fold 에 대해서 한 class의 표준편차를 보는것(데이터 퍼진정도를 보는것)
            class_std = np.std([class_counts_per_fold[i][class_number]/class_counts_total[class_number] for i in range(k)])
            std_per_class.append(class_std)
        # std_per_class는 class 별로 표준편차를 나타내줌
        class_counts_per_fold[fold] -= class_counts_image # fold에 한번 넣어서 표준편차를 구해본것-> 다시 원상복구(모든 fold에 넣어보고 그중 fold별로 class 분포의 표준편차가 제일 작은 값인 fold 에 그 이미지를 넣을것임
        
        # 한 fold에 어떤 클래스가 몰리게 되면 fold에 대한 그 클래스에 대한 표준편차는 커질것 -> 모든 클래스에 대해 고르게 분포해야하므로 전체 클래스에 대한 표준편차들의 평균을 이용
        return np.mean(std_per_class) # 평균([fold별 0클래스에 대한 표준편차,fold별 1클래스에 대한 표준편차,...,fold별 10클래스에 대한 표준편차])
    
    image_names_and_class_counts = list(class_counts_per_image_names.items()) # [('image_name',[0,0,0,0,...,0]),('image_name',[0,0,0...,0]),...]
    random.Random(seed).shuffle(image_names_and_class_counts) # 리스트를 랜덤하게 섞어줌
    
    # 한 이미지의 class_counts의 표준편차가 큰거(한 클래스에 많은 값이 몰려있을수 있기 때문에 미리 처리)부터 fold 별 분류 시작
    for image_name,class_counts in sorted(image_names_and_class_counts,key=lambda x:-np.std(x[1])):
        best_fold = None
        min_eval = None
        for i in range(k): # 한 개의 이미지를 k개의 fold에 다 넣어보고 그중 가장 fold별로 class 분포가 고르게 퍼지게 할수있는 fold에 그 이미지를 넣어줌
            fold_eval = eval_class_counts_per_fold(class_counts,i)
            if min_eval is None or fold_eval < min_eval:
                min_eval = fold_eval
                best_fold = i
        class_counts_per_fold[best_fold] += class_counts # fold에 이제 삽입이 되었으므로 그 fold에 그 이미지에 해당하는 class 분포를 넣어줌
        image_names_per_fold[best_fold].add(image_name) # image_name을 fold별로 기록
    
    all_image_names = set(image_names) # 중복되지 않는 이미지 이름들 , 즉 전체 이미지 개수만큼 있음
    
    for i in range(k):
        train_image_names = all_image_names - image_names_per_fold[i]
        test_image_names = image_names_per_fold[i]
        
        # image_names 는 전체 annotations 개수 , 중복되는 이미지이름 존재
        # df 에서 image_name이 우리가 이미지별 클래스 분포에 따라 분류한 i번째 fold에 있다면 인덱스 번호를 저장 
        train_indices = [i for i,image_name in enumerate(image_names) if image_name in train_image_names]
        test_indices = [i for i,image_name in enumerate(image_names) if image_name in test_image_names]
        # 즉 df 에서 이미지 이름이 같다면 다 같은 fold 일것임 (우리는 이미지에 따라 분류를 했기 때문)
        
        yield train_indices,test_indices # df의 index 번호들이 저장되어져있음

In [20]:
def get_folds(df, config):
    df_folds = df[['image_id']].copy() ## df[['image_id']] 괄호가 2개이면 Dataframe 한개면 Series로 지정됨
    df_folds.loc[:, 'bbox_count'] = 1
    # 여기까지는 중복되는 이미지가 포함 , annotation 정보에 따라 만들어진 df
    
    # groupby count을 통해 중복되는 데이터가 몇개인지 표시 , groupby 했으므로 index가 image_id로 변경됨
    df_folds = df_folds.groupby('image_id').count() # 이미지별로 객체가 몇개있는지 파악
    # fold 번호 지정해주기 위한 열 생성
    df_folds['fold'] = 0
    
    for fold, (trn_idx, val_idx) in enumerate(
            stratified_k_fold(df, df['class_id'], df['image_id'], config.n_folds, config.seed)):
        
        # df의 인덱스를 지정해주고 그 인덱스와 image_id를 지정해주면 그에 맞는 값들이 나오고 unique를 통해서 리스트 형태로 만들어주기
        trn_ids = df.loc[trn_idx, 'image_id'].unique() # unique 한값을 ['batch_01_vt/0003.jpg' ,...,'batch_03/1000.jpg'] 처럼 리스트로 만들어줌
        val_ids = df.loc[val_idx, 'image_id'].unique()
        assert len(set(trn_ids).intersection(set(val_ids))) == 0 # 이 조건을 만족안하면 train하고 valid가 겹치는것이 있으므로 잘못된것!

        df_folds.loc[val_ids, 'fold'] = fold # valid에 fold번호를 붙여줌
    return df_folds # 이미지별 fold 정보가 저장된 df 반환

In [21]:
def load_all_data():
    meta_df = pd.read_csv(f"./data.csv")
    meta_df = meta_df.reset_index(drop=True)

    return meta_df

In [22]:
meta_df = load_all_data()
meta_df['class_id']=meta_df['class_id'].astype(int)

seed_everything()
f_folds = get_folds(meta_df,CFG)

In [23]:
print(f_folds)

                      bbox_count  fold
image_id                              
batch_01_vt/0002.jpg          17     0
batch_01_vt/0003.jpg          14     3
batch_01_vt/0005.jpg           1     4
batch_01_vt/0006.jpg           2     4
batch_01_vt/0007.jpg           2     1
...                          ...   ...
batch_03/0994.jpg              7     4
batch_03/0995.jpg             14     0
batch_03/0996.jpg              4     2
batch_03/0997.jpg              8     2
batch_03/1000.jpg              1     4

[3272 rows x 2 columns]


In [24]:
f_folds = f_folds.reset_index()

In [25]:
print(f_folds)

                  image_id  bbox_count  fold
0     batch_01_vt/0002.jpg          17     0
1     batch_01_vt/0003.jpg          14     3
2     batch_01_vt/0005.jpg           1     4
3     batch_01_vt/0006.jpg           2     4
4     batch_01_vt/0007.jpg           2     1
...                    ...         ...   ...
3267     batch_03/0994.jpg           7     4
3268     batch_03/0995.jpg          14     0
3269     batch_03/0996.jpg           4     2
3270     batch_03/0997.jpg           8     2
3271     batch_03/1000.jpg           1     4

[3272 rows x 3 columns]


In [29]:
from tqdm import tqdm

# 기존 형태를 가져오기 위해 json 파일 load
dataset_path = './data'
annotations_file_path = dataset_path + '/' + 'train_all.json'

with open(annotations_file_path,'r') as f:
    dataset = json.loads(f.read())
    

# 똑같이 json 형태로 만들어주는 과정
for idx in tqdm(range(5)):
    images = []
    annotations = []
    
    # fold별 train,valid 이미지 name list로 만들기
    train_file_names = f_folds[f_folds['fold']!=idx]['image_id'].unique() # 리스트로 만들어주기
    valid_file_names = f_folds[f_folds['fold']==idx]['image_id'].unique()
    
    print(len(train_file_names))
    print(len(valid_file_names))
    
    # 기존 json 형태보고 맞춰서 작성하면 됨
    for i,train_file_name in enumerate(train_file_names):
        images.append(dict(
            license=0,
            url=None,
            file_name=train_file_name,
            height=512,
            width=512,
            date_captured=None,
            id=i)) # 새롭게 이미지 id 번호 지정
        
        # image_id 를 file_name을 통해서 찾기 image_id=[image_id] 한개의 원소로 이루어짐 [0]으로 부르고 그 중 image id 가져오기
        image_id = list(filter(lambda x:x['file_name']==train_file_name,dataset['images']))[0]['id']
        # annotations 파일중에 image_id가 image_id랑 같은 것들이 있다면 (한개의 이미지안에 들어있는 한개또는 여러개의 객체리스트)
        for x in list(filter(lambda x:x['image_id']==image_id,dataset['annotations'])):
            annotations.append(dict(id=len(annotations), # annotations id 를 새로 0번부터 지정
                                   image_id=i,
                                   category_id=x['category_id'],
                                   segmentation=x['segmentation'],
                                   area=x['area'],
                                   bbox=x['bbox'],
                                   iscrows=x['iscrowd']))
    
    train_ann={}
    train_ann['images']=images
    train_ann['annotations']=annotations
    train_ann['categories']=dataset['categories']
    
    with open(f'train_data{idx}.json','w') as f:
        json.dump(train_ann,f,indent=4)
        
    images = []
    annotations = []
    for i, valid_file_name in (enumerate(valid_file_names)):
        images.append(dict(
                license=0,
                url=None,
                file_name=valid_file_name,
                height=512,
                width=512,
                date_captured=None,
                id=i
            ))

        image_id = list(filter(lambda x: x['file_name'] == valid_file_name, dataset['images']))[0]['id']
        for x in list(filter(lambda x: x['image_id'] == image_id, dataset['annotations'])):
            annotations.append(dict(id=len(annotations), 
                                image_id=i, 
                                category_id=x['category_id'], 
                                segmentation=x['segmentation'],
                                area=x['area'], 
                                bbox=x['bbox'], 
                                iscrowd=x['iscrowd']))
    
    valid_ann = {}
    valid_ann['images'] =  images
    valid_ann['annotations'] = annotations
    valid_ann['categories'] = dataset['categories']
        
    with open(f'valid_data{idx}.json', 'w') as f:
        json.dump(valid_ann, f, indent=4)

  0%|          | 0/5 [00:00<?, ?it/s]

2617
655


 20%|██        | 1/5 [00:13<00:54, 13.54s/it]

2611
661


 40%|████      | 2/5 [00:27<00:40, 13.57s/it]

...

 60%|██████    | 3/5 [00:40<00:27, 13.63s/it]

2625
647


 80%|████████  | 4/5 [00:54<00:13, 13.71s/it]

2622
650


100%|██████████| 5/5 [01:08<00:00, 13.64s/it]
